In [1]:
import numpy as np
import pandas as pd
import scanpy as sc

In [2]:
datapath='/ahg/regevdata/projects/Cell2CellCommunication/perturbseq_benchmarks/data/2018-11-09'
dataset='dc_3hr'
gsm_number='GSM2396856'
anno=datapath+'/'+dataset+'/'+gsm_number+'_'+dataset+'_cbc_gbc_dict_lenient.csv.gz' #also experiment with the strict
pref=datapath+'/'+dataset+'/'+dataset

In [3]:
mtx_file=datapath+'/'+dataset+'/GSM2396856_dc_3hr.mtx.txt.gz'
adata=sc.read_mtx(mtx_file)
num_cells=len(adata.obs_names)
num_genes=len(adata.var_names)

In [4]:
adata

AnnData object with n_obs × n_vars = 17775 × 32777 

In [5]:
print('Attaching gene names, cell names')
cellnames=datapath+'/'+dataset+'/GSM2396856_dc_3hr_cellnames.csv.gz'
genenames=datapath+'/'+dataset+'/GSM2396856_dc_3hr_genenames.csv.gz'
print(genenames)
cells=list(pd.read_csv(cellnames).iloc[:,1])
genes=list(pd.read_csv(genenames).iloc[:,1])
print(genes[:5])

Attaching gene names, cell names
/ahg/regevdata/projects/Cell2CellCommunication/perturbseq_benchmarks/data/2018-11-09/dc_3hr/GSM2396856_dc_3hr_genenames.csv.gz
['ENSMUSG00000033845_Mrpl15', 'ENSMUSG00000025903_Lypla1', 'ENSMUSG00000033813_Tcea1', 'ENSMUSG00000002459_Rgs20', 'ENSMUSG00000033793_Atp6v1h']


In [6]:
genes2=[]
for i in range(len(genes)):
    genes2.append(genes[i].split('_')[1])
print(genes2[:4])

['Mrpl15', 'Lypla1', 'Tcea1', 'Rgs20']


In [7]:
if len(cells)==num_cells and len(genes2)==num_genes:
    adata.obs_names=cells
    adata.var_names=genes2
elif len(cells)==num_genes and len(genes2)==num_cells:
    #need to transpose adata
    adata=adata.T
    adata.obs_names=cells
    adata.var_names=genes2

In [8]:
adata.var_names[:10]

Index(['Mrpl15', 'Lypla1', 'Tcea1', 'Rgs20', 'Atp6v1h', 'Rb1cc1',
       '4732440D04Rik', 'St18', 'Pcmtd1', 'Gm26901'],
      dtype='object')

In [9]:
#some dataset-specific things
#============================

#annotate batch
batch_var=[]
for i in range(adata.n_obs):
    cell=adata.obs_names[i]
    batch_here=cell.split('_')[2]
    batch_var.append(batch_here)
adata.obs['batch']=batch_var

In [10]:
set(adata.obs['batch'])

{'A8', 'A9', 'B8', 'B9', 'C8', 'C9', 'D8', 'D9'}

In [11]:
adata

AnnData object with n_obs × n_vars = 32777 × 17775 
    obs: 'batch'

In [12]:
adata.write(pref+'raw_counts.h5ad')

... storing 'batch' as categorical


In [13]:
#annotate perturbations in each cell
import re
import gzip

gbc_cbc={}
for line in gzip.open(anno,'r').readlines():
    line_text=line.decode('utf-8')
    items=re.sub(' ','',re.sub('"','',line_text.strip())).split(',')
    g=items[0]
    gbc_cbc[g]=items  

In [14]:
for g in gbc_cbc.keys():
    print(g)
    vals=np.zeros((len(adata.obs_names),1))
    for cell_idx in range(len(adata.obs_names)):
        cell=str(adata.obs_names[cell_idx])
        if cell in gbc_cbc[g]:
            vals[cell_idx]=1
    adata.obs[g]=vals

m_Egr1_4
m_Irf1_4
m_Irf1_2
m_Irf1_1
m_Hif1a_3
m_Cebpb_1
m_Hif1a_1
m_Cebpb_3
m_Hif1a_4
m_Maff_1
m_Maff_4
m_Irf4_3
m_Stat1_3
m_Atf3_2
m_Junb_4
m_E2f1_3
m_Relb_1
m_Ets2_3
m_Ahr_1
m_Ets2_4
m_Ahr_3
m_E2f1_4
m_Runx1_4
m_Atf3_1
m_Runx1_2
m_Rel_3
m_Rel_2
m_Rel_1
m_Stat1_2
m_Stat2_4
m_Spi1_4
m_Spi1_2
m_Stat1_1
m_Stat2_2
m_Spi1_3
m_Nfkb1_3
m_Stat3_3
m_E2f4_4
m_Stat2_3
m_E2f4_2
m_E2f4_3
m_Irf4_2
m_Ctcf_2
m_Ctcf_1
m_Egr2_2
m_Nfkb1_4
m_Egr2_4
m_Irf4_4
m_Rela_3
m_MouseNTC_100_A_67005
m_Rela_2
m_Irf2_1
m_Irf2_2
m_Irf2_3
m_Irf2_4
m_Rela_1
m_Nfkb1_2


Create input files
===

In [15]:
adata.obs_names[:10]

Index(['AAACATACACGTAC_dc3hLPS_A8', 'AAACATACATGTCG_dc3hLPS_A8',
       'AAACATACCAACTG_dc3hLPS_A8', 'AAACATACTCCTTA_dc3hLPS_A8',
       'AAACATACTCTCCG_dc3hLPS_A8', 'AAACATTGCTGTCC_dc3hLPS_A8',
       'AAACATTGCTTATC_dc3hLPS_A8', 'AAACATTGGCGGAA_dc3hLPS_A8',
       'AAACATTGTACTGG_dc3hLPS_A8', 'AAACCGTGCACCAA_dc3hLPS_A8'],
      dtype='object')

In [16]:
#cell2guide
guides=['m_Egr1_4', 'm_Irf1_4', 'm_Irf1_2', 'm_Irf1_1', 'm_Hif1a_3', 'm_Cebpb_1', 'm_Hif1a_1', 'm_Cebpb_3', 'm_Hif1a_4', 'm_Maff_1', 'm_Maff_4', 'm_Irf4_3', 'm_Stat1_3', 'm_Atf3_2', 'm_Junb_4', 'm_E2f1_3', 'm_Relb_1', 'm_Ets2_3', 'm_Ahr_1', 'm_Ets2_4', 'm_Ahr_3', 'm_E2f1_4', 'm_Runx1_4', 'm_Atf3_1', 'm_Runx1_2', 'm_Rel_3', 'm_Rel_2', 'm_Rel_1', 'm_Stat1_2', 'm_Stat2_4', 'm_Spi1_4', 'm_Spi1_2', 'm_Stat1_1', 'm_Stat2_2', 'm_Spi1_3', 'm_Nfkb1_3', 'm_Stat3_3', 'm_E2f4_4', 'm_Stat2_3', 'm_E2f4_2', 'm_E2f4_3', 'm_Irf4_2', 'm_Ctcf_2', 'm_Ctcf_1', 'm_Egr2_2', 'm_Nfkb1_4', 'm_Egr2_4', 'm_Irf4_4', 'm_Rela_3', 'm_MouseNTC_100_A_67005', 'm_Rela_2', 'm_Irf2_1', 'm_Irf2_2', 'm_Irf2_3', 'm_Irf2_4', 'm_Rela_1', 'm_Nfkb1_2']
cell2guide_df=adata.obs[guides]
cell2guide_df['cell']=list(cell2guide_df.index)
cell2guide_df.to_csv(pref+'.cell2guide.csv.gz',sep='\t',compression='gzip',index=None)

In [17]:
#guide2gene
guide2gene_df=pd.DataFrame({'guide':guides})
guide2gene_df['gene']='NA'
guide2gene_df.index=list(guide2gene_df['guide'])

import re
for g in guides:
    guide2gene_df.loc[g,'gene']=re.sub('m_','',g).split('_')[0]
print(guide2gene_df)
guide2gene_df.to_csv(pref+'.guide2gene.csv.gz',sep='\t',compression='gzip',index=None)
print(pref+'.guide2gene.csv.gz')

                                         guide      gene
m_Egr1_4                              m_Egr1_4      Egr1
m_Irf1_4                              m_Irf1_4      Irf1
m_Irf1_2                              m_Irf1_2      Irf1
m_Irf1_1                              m_Irf1_1      Irf1
m_Hif1a_3                            m_Hif1a_3     Hif1a
m_Cebpb_1                            m_Cebpb_1     Cebpb
m_Hif1a_1                            m_Hif1a_1     Hif1a
m_Cebpb_3                            m_Cebpb_3     Cebpb
m_Hif1a_4                            m_Hif1a_4     Hif1a
m_Maff_1                              m_Maff_1      Maff
m_Maff_4                              m_Maff_4      Maff
m_Irf4_3                              m_Irf4_3      Irf4
m_Stat1_3                            m_Stat1_3     Stat1
m_Atf3_2                              m_Atf3_2      Atf3
m_Junb_4                              m_Junb_4      Junb
m_E2f1_3                              m_E2f1_3      E2f1
m_Relb_1                       